In [2]:
string name = "Michael";

name

Michael

# Identity and Stuff

Hi there!

In [3]:
record Person(string name, DateTime dateOfBirth) { }

var michael = new Person("Michael Perry", new DateTime(1971, 5, 10));

michael

name,Michael Perry
dateOfBirth,1971-05-10 00:00:00Z


In [4]:
int AgeOfPerson(Person person)
{
  var today = DateTime.Today;
  var age = today.Year - person.dateOfBirth.Year;
  if (person.dateOfBirth.Date > today.AddYears(-age)) age--;
  return age;
}

AgeOfPerson(michael)

53

In [5]:
using System.Collections.Immutable;

var letters = ImmutableArray.Create('a', 'b', 'c');

letters

[ a, b, c ]

In [6]:
var newArray = letters.Add('d');
var moreLetters = ImmutableArray<char>.Empty.Add('e').Add('f').Add('g');

moreLetters

[ e, f, g ]

In [7]:
newArray

[ a, b, c, d ]

In [8]:
#r "./TicTacToe/bin/Debug/net7.0/TicTacToe.dll"

using TicTacToe;

var game = Game.Empty
  .Play(4);

game.Html

<!DOCTYPE html>
 
 
 Tic-Tac-Toe Board 
 
 
 
 X

In [9]:
var moves = game.EmptySquares.Select(index =>
  game.Play(index).Html
);

moves

index,value
0,<!DOCTYPE html> Tic-Tac-Toe Board OX
1,<!DOCTYPE html> Tic-Tac-Toe Board OX
2,<!DOCTYPE html> Tic-Tac-Toe Board OX
3,<!DOCTYPE html> Tic-Tac-Toe Board OX
4,<!DOCTYPE html> Tic-Tac-Toe Board XO
5,<!DOCTYPE html> Tic-Tac-Toe Board XO
6,<!DOCTYPE html> Tic-Tac-Toe Board XO
7,<!DOCTYPE html> Tic-Tac-Toe Board XO


In [10]:
game = game.Play(1);

game.Html

<!DOCTYPE html>
 
 
 Tic-Tac-Toe Board 
 
 
 
 O X

In [11]:
Symbol Evaluate(Game game)
{
  var winner = game.Winner;
  if (winner != Symbol.Empty)
    return winner;
  if (!game.EmptySquares.Any())
    return Symbol.Empty;

  var outcomes = game.EmptySquares.Select(index =>
    Evaluate(game.Play(index))
  ).ToImmutableArray();
  if (outcomes.Any(outcome => outcome == game.NextPlayer))
    return game.NextPlayer;
  if (outcomes.Any(outcome => outcome == Symbol.Empty))
    return Symbol.Empty;
  return game.NextPlayer == Symbol.X ? Symbol.O : Symbol.X;
}

Evaluate(game)

X

In [12]:
game.EmptySquares.Select(index =>
  game.Play(index)
).Select(game =>
  game.HtmlWithOutcome(Evaluate(game))
)

index,value
0,<!DOCTYPE html> Tic-Tac-Toe Board XOXX wins
1,<!DOCTYPE html> Tic-Tac-Toe Board OXXX wins
2,<!DOCTYPE html> Tic-Tac-Toe Board OXXX wins
3,<!DOCTYPE html> Tic-Tac-Toe Board OXXX wins
4,<!DOCTYPE html> Tic-Tac-Toe Board OXXX wins
5,<!DOCTYPE html> Tic-Tac-Toe Board OXXDraw
6,<!DOCTYPE html> Tic-Tac-Toe Board OXXX wins


## SQL Database

```sql
DROP TABLE IF EXISTS Course;

CREATE TABLE Course (
  CourseID int IDENTITY(1,1) NOT NULL,
  Code nvarchar(10) NOT NULL,
  Title nvarchar(50) NOT NULL,
  Credits int NOT NULL,
  CONSTRAINT PK_Course PRIMARY KEY (CourseID)
);

CREATE UNIQUE INDEX IX_Course_Code ON Course (Code);
```

```sql
INSERT INTO Course (Title, Code, Credits)
SELECT 'Chemistry', 'CHEM-101', 3
WHERE NOT EXISTS (SELECT * FROM Course WHERE Code = 'CHEM-101')
```

In [19]:
#r "nuget: Jinaga"
#r "nuget: Jinaga.Notebooks"

using Jinaga;
using Jinaga.Notebooks;

Installed Packages Jinaga, 1.0.6 Jinaga.Notebooks, 1.0.5

In [34]:
[FactType("School.Institution")]
public record Institution(string identifier) {}

[FactType("School.Course")]
public record Course(Institution institution, string code) {}

[FactType("School.Course.Title")]
public record CourseTitle(Course course, string title, CourseTitle[] prior) {}

[FactType("School.Course.Credits")]
public record CourseCredits(Course course, int credits, CourseCredits[] prior) {}

Renderer.RenderTypes(typeof(Course), typeof(CourseTitle), typeof(CourseCredits))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.0.0 (20240704.0754)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- School.Course -->
 
 School.Course 
 
 School.Course 
 
<!-- School.Institution -->
 
 School.Institution 
 
 School.Institution 
 
<!-- School.Course->School.Institution -->
 
 School.Course->School.Institution 
 
 
 institution 
 
<!-- School.Course.Title -->
 
 School.Course.Title 
 
 School.Course.Title 
 
<!-- School.Course.Title->School.Course -->
 
 School.Course.Title->School.Course 
 
 
 course 
 
<!-- School.Course.Title->School.Course.Title -->
 
 School.Course.Title->School.Course.Title 
 
 
 prior 
 
<!-- School.Course.Credits -->
 
 School.Course.Credits 
 
 School.Course.Credits 
 
<!-- School.Course.Credits->School.Course -->
 
 School.Course.Credits->School.Course 
 
 
 course 
 
<!-- School.Course.Credits->School.Course.Credits -->
 
 School.Course.Credits->School.Course.Credits 
 
 
 prior

In [35]:
var j = JinagaClient.Create();

var universityOfTulsa = await j.Fact(new Institution("03185"));
var trigonometry = await j.Fact(new Course(universityOfTulsa, "MATH-210"));
var trigonometryTitle = await j.Fact(new CourseTitle(trigonometry, "Trigonometry", new CourseTitle[] { }));
var trigTitle = await j.Fact(new CourseTitle(trigonometry, "Trig", new CourseTitle[] { trigonometryTitle }));
var otherTitle = await j.Fact(new CourseTitle(trigonometry, "Other", new CourseTitle[] { trigonometryTitle }));

j.RenderFacts(trigonometry, trigonometryTitle, trigTitle, otherTitle)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.0.0 (20240704.0754)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- BiypzObEuhT2FVxThR9EwxcwMl45cWumcPZ1I9/w9tOy19UzPn6cRDVnT9tB0fiqGhVlAG4Z4zoDloZRVm3Hlw== -->
 
 BiypzObEuhT2FVxThR9EwxcwMl45cWumcPZ1I9/w9tOy19UzPn6cRDVnT9tB0fiqGhVlAG4Z4zoDloZRVm3Hlw== 
 
 School.Institution 
 
 identifier 
 
 03185 
 
<!-- yIMl54nT/Cuschj0+1wFgMcSvhAsuBcevCnnNk9akjB4Pwy1bKZZ10g0eMQWnvPjf/V2m0xsJPx930t3eX41oA== -->
 
 yIMl54nT/Cuschj0+1wFgMcSvhAsuBcevCnnNk9akjB4Pwy1bKZZ10g0eMQWnvPjf/V2m0xsJPx930t3eX41oA== 
 
 School.Course 
 
 code 
 
 MATH-210 
 
 
<!-- yIMl54nT/Cuschj0+1wFgMcSvhAsuBcevCnnNk9akjB4Pwy1bKZZ10g0eMQWnvPjf/V2m0xsJPx930t3eX41oA==->BiypzObEuhT2FVxThR9EwxcwMl45cWumcPZ1I9/w9tOy19UzPn6cRDVnT9tB0fiqGhVlAG4Z4zoDloZRVm3Hlw== -->
 
 yIMl54nT/Cuschj0+1wFgMcSvhAsuBcevCnnNk9akjB4Pwy1bKZZ10g0eMQWnvPjf/V2m0xsJPx930t3eX41oA==->BiypzObEuhT2FVxThR9EwxcwMl45cWumcPZ1I9/w9tOy19UzPn6cRDVnT9tB0fiqGhVlAG4Z4zoDloZRVm3Hlw== 
 
 
 institution 
 
<!-- RGI5z2F31AAXQIyZBjp2QTiTejWVmzyQoCZ+UdQJJHwZwlfEqvNMzXwXJvHXzhrJ/OC1nyvF44c8OrJzvF0OEg== -->
 
 RGI5z2F31AAXQIyZBjp2QTiTejWVmzyQoCZ+UdQJJHwZwlfEqvNMzXwXJvHXzhrJ/OC1nyvF44c8OrJzvF0OEg== 
 
 School.Course.Title 
 
 title 
 
 Trigonometry 
 
 
<!-- RGI5z2F31AAXQIyZBjp2QTiTejWVmzyQoCZ+UdQJJHwZwlfEqvNMzXwXJvHXzhrJ/OC1nyvF44c8OrJzvF0OEg==->yIMl54nT/Cuschj0+1wFgMcSvhAsuBcevCnnNk9akjB4Pwy1bKZZ10g0eMQWnvPjf/V2m0xsJPx930t3eX41oA== -->
 
 RGI5z2F31AAXQIyZBjp2QTiTejWVmzyQoCZ+UdQJJHwZwlfEqvNMzXwXJvHXzhrJ/OC1nyvF44c8OrJzvF0OEg==->yIMl54nT/Cuschj0+1wFgMcSvhAsuBcevCnnNk9akjB4Pwy1bKZZ10g0eMQWnvPjf/V2m0xsJPx930t3eX41oA== 
 
 
 course 
 
<!-- QU65ksoB7zlsXRr2DgwyZMCFwd/xM4jlO1tdfPLhQIGEe3Ir8ROVdnWt4IYdATENdbVmQ7mUHfbKeXdQjGwiSQ== -->
 
 QU65ksoB7zlsXRr2DgwyZMCFwd/xM4jlO1tdfPLhQIGEe3Ir8ROVdnWt4IYdATENdbVmQ7mUHfbKeXdQjGwiSQ== 
 
 School.Course.Title 
 
 title 
 
 Trig 
 
 
<!-- QU65ksoB7zlsXRr2DgwyZMCFwd/xM4jlO1tdfPLhQIGEe3Ir8ROVdnWt4IYdATENdbVmQ7mUHfbKeXdQjGwiSQ==->yIMl54nT/Cuschj0+1wFgMcSvhAsuBcevCnnNk9akjB4Pwy1bKZZ10g0eMQWnvPjf/V2m0xsJPx930t3eX41oA== -->
 
 QU65ksoB7zlsXRr2DgwyZMCFwd/xM4jlO1tdfPLhQIGEe3Ir8ROVdnWt4IYdATENdbVmQ7mUHfbKeXdQjGwiSQ==->yIMl54nT/Cuschj0+1wFgMcSvhAsuBcevCnnNk9akjB4Pwy1bKZZ10g0eMQWnvPjf/V2m0xsJPx930t3eX41oA== 
 
 
 course 
 
<!-- QU65ksoB7zlsXRr2DgwyZMCFwd/xM4jlO1tdfPLhQIGEe3Ir8ROVdnWt4IYdATENdbVmQ7mUHfbKeXdQjGwiSQ==->RGI5z2F31AAXQIyZBjp2QTiTejWVmzyQoCZ+UdQJJHwZwlfEqvNMzXwXJvHXzhrJ/OC1nyvF44c8OrJzvF0OEg== -->
 
 QU65ksoB7zlsXRr2DgwyZMCFwd/xM4jlO1tdfPLhQIGEe3Ir8ROVdnWt4IYdATENdbVmQ7mUHfbKeXdQjGwiSQ==->RGI5z2F31AAXQIyZBjp2QTiTejWVmzyQoCZ+UdQJJHwZwlfEqvNMzXwXJvHXzhrJ/OC1nyvF44c8OrJzvF0OEg== 
 
 
 prior 
 
<!-- /+gQVHsHLBKZ8oPvxwmjAZA5hgwB9F6NujKwHCzremL4i0KIAbZg98WqZEoa7b+BwSt5zPfZPZxYYlThMZzl5Q== -->
 
 /+gQVHsHLBKZ8oPvxwmjAZA5hgwB9F6NujKwHCzremL4i0KIAbZg98WqZEoa7b+BwSt5zPfZPZxYYlThMZzl5Q== 
 
 School.Course.Title 
 
 title 
 
 Other 
 
 
<!-- /+gQVHsHLBKZ8oPvxwmjAZA5hgwB9F6NujKwHCzremL4i0KIAbZg98WqZEoa7b+BwSt5zPfZPZxYYlThMZzl5Q==->yIMl54nT/Cuschj0+1wFgMcSvhAsuBcevCnnNk9akjB4Pwy1bKZZ10g0eMQWnvPjf/V2m0xsJPx930t3eX41oA== -->
 
 /+gQVHsHLBKZ8oPvxwmjAZA5hgwB9F6NujKwHCzremL4i0KIAbZg98WqZEoa7b+BwSt5zPfZPZxYYlThMZzl5Q==->yIMl54nT/Cuschj0+1wFgMcSvhAsuBcevCnnNk9akjB4Pwy1bKZZ10g0eMQWnvPjf/V2m0xsJPx930t3eX41oA== 
 
 
 course 
 
<!-- /+gQVHsHLBKZ8oPvxwmjAZA5hgwB9F6NujKwHCzremL4i0KIAbZg98WqZEoa7b+BwSt5zPfZPZxYYlThMZzl5Q==->RGI5z2F31AAXQIyZBjp2QTiTejWVmzyQoCZ+UdQJJHwZwlfEqvNMzXwXJvHXzhrJ/OC1nyvF44c8OrJzvF0OEg== -->
 
 /+gQVHsHLBKZ8oPvxwmjAZA5hgwB9F6NujKwHCzremL4i0KIAbZg98WqZEoa7b+BwSt5zPfZPZxYYlThMZzl5Q==->RGI5z2F31AAXQIyZBjp2QTiTejWVmzyQoCZ+UdQJJHwZwlfEqvNMzXwXJvHXzhrJ/OC1nyvF44c8OrJzvF0OEg== 
 
 
 prior

In [36]:
var coursesAtInstitution = Given<Institution>.Match((institution, facts) =>
  from course in facts.OfType<Course>()
  where course.institution == institution
  select new
  {
    course,
    title = facts.OfType<CourseTitle>()
      .Where(title => title.course == course &&
        !facts.Any<CourseTitle>(next => next.prior.Contains(title))
      )
      .Select(title => title.title),
    credits = facts.OfType<CourseCredits>()
      .Where(credits => credits.course == course &&
        !facts.Any<CourseCredits>(next => next.prior.Contains(credits))
      )
      .Select(credits => credits.credits)
  }
);

var courses = await j.Query(coursesAtInstitution, universityOfTulsa);
courses

index value 0 { course = Course { institution = Institution { identifier = 03185 }, code = MATH-210 }, title = System.Linq.Enumerable+<OfTypeIterator>d__66`1[System.String], credits = System.Linq.Enumerable+<OfTypeIterator>d__66`1[System.Int32] } course Course { institution = Institution { identifier = 03185 }, code = MATH-210 } institution Institution { identifier = 03185 } identifier 03185 code MATH-210 title [ Trig, Other ] (values) [ Trig, Other ] credits [ ] (values) [ ]